# SQL

O SQL usa os termos **tabela**, **linha** e **coluna** para os termos do modelo relacional formal *relação*, *tupla* e *atributo* respectivamente.

In [1]:
# Utilizaremos a lib csvms em nossos exemplos
from csvms.table import Table
#For re-run
import shutil
from csvms.schema import FILE_DIR
shutil.rmtree(FILE_DIR, ignore_errors=True)

INFO:CSVMS:version:0.2.2


## Criação

O comando `CREATE TABLE` é usado para especificar uma nova relação, dando-lhe um **nome** e especificando seus **atributos**. Cada um dos atributos recebe um **nome** e um **tipo de dado** para especificar seu domínio de valores.

> Como nosso banco de dados será escrito na linguagem [python](https://www.w3schools.com/python/default.asp), utilizaremos os seus [tipos primitivos](https://www.w3schools.com/python/python_datatypes.asp) como tipos de dados.

Os tipos de dados básicos disponíveis para atributos costumam ser:

+ Numéricos
+ Cadeia de caracters
+ Cadeia de bits
+ Booleano
+ Data
+ Hora

> E é já a partir deste ponto em que os vários dialetos começam a surgir 😟

```sql
CREATE TABLE dept (
    codigo INTEGER ,
    descricao TEXT);
```

In [2]:
Table(                   #Criação da tabela 'departamento'
    name="dept",         #Definição do nome da tabela
    columns={            #Atributos da tabela com seus respectivos nomes e tipos
        'codigo': int,   #Criaremos um atributo para armazenar 
        'descricao':str} #Departamento armazenará o código do departamento
).save()                 #Salva as definições da tabela no catalogo de dados

INFO:CSVMS:creating new data dictionary in data/catalog.json


True

## Transação

O inicio do trabalho `BEGIN` se dá com o carregamento da tabela do sistema de arquivos para a SGA (System Global Area).
Tudo o que ocorrer com este objeto durante seu período em mória será considerado uma transação até que se confirme com os comandos de `COMMIT` ou `ROLLBACK`

In [3]:
sga = dict(
    dept=Table("dept")
)
print(sga['dept'])

   TABLE: default.dept
   +-------+----------+
   |codigo |descricao |
   +-------+----------+
   +-------+----------+



## Inclusão

Em sua forma mais simples, o comando `INSERT` é usado para acrescentar uma única *tupla* (linha) em uma *relação* (tabela).
Precisamos especificar o nome da relação e uma lista de valores para a tupla. Os valores devem ser informados na mesma ordem em que os atributos correspondentes foram especificados no `CREATE TABLE`.

```sql
INSERT INTO dept VALUES (1, "Compras");
INSERT INTO dept VALUES (2, "Vendas");
INSERT INTO dept VALUES (3, "Suporte");
```

In [4]:

sga['dept'].append(1, "Compras")
sga['dept'].append(2, "Vendas")
sga['dept'].append(3, "Suporte")
print(sga['dept'])

INFO:CSVMS:Row inserted
INFO:CSVMS:Row inserted
INFO:CSVMS:Row inserted


   TABLE: default.dept
   +-------+----------+
   |codigo |descricao |
   +-------+----------+
  0|      1|   Compras|
  1|      2|    Vendas|
  2|      3|   Suporte|
   +-------+----------+



## Confirmação

Com o comando de `COMMIT` confirmamos que todas as alterações desta transação deverão ser **salvas** em definitivo.

```sql
COMMIT
```

In [5]:
# Salva o estado de todas as tabelas da SGA
for tbl in sga.values():
    tbl.save()

## Exclusão

O comando `DELETE` remove *tuplas* de uma relação. Ele possui uma clausula *`WHERE`*, semelhante a utilzada em consultas, para selecionar a tuplas que serão excluídas.

```sql
DELETE FROM dept WHERE descricao = 'Vendas'
```

In [6]:
# Procura em toda a tabela por linhas cuja descrição seja "Vendas"
for idx in range(len(sga['dept'])):
    if sga['dept'][idx]["descricao"]=="Vendas":
        del sga['dept'][idx]
print(sga['dept'])

INFO:CSVMS:Row deleted


   TABLE: default.dept
   +-------+----------+
   |codigo |descricao |
   +-------+----------+
  0|      1|   Compras|
  1|      3|   Suporte|
   +-------+----------+



## Atualização

O comando `UPDATE` é utilizado para modificar valores de atributos de uma ou mais *tuplas* selecionadas. Assim como no comando `DELETE`, é possível adicionar uma clausula `WHERE` no comando para informar o critério de seleção das linhas que serão afetadas.

```sql
UPDATE dept 
   SET descricao = 'Financeiro'
 WHERE codigo = 3
```

In [7]:
for idx in range(len(sga['dept'])):
    if sga['dept'][idx]["codigo"]==3:
        row = sga['dept'][idx]
        row["descricao"]='Financeiro'
        sga['dept'][idx] = tuple(row.values())
print(sga['dept'])

INFO:CSVMS:Row updated


   TABLE: default.dept
   +-------+----------+
   |codigo |descricao |
   +-------+----------+
  0|      1|   Compras|
  1|      3|Financeiro|
   +-------+----------+



## Descarte

Com o comando de `ROLLBACK` descartamos todas as alterações desta transação, voltando assim ao estado em que os dados se encontravam desde a ultima operação salva. O comando de `ROLLBACK` finalizará um trabalho e iniciará outro.

```sql
ROLLBACK
```

In [8]:
# Recarrega todas as tabelas da SGA
for tbl in sga:
    sga[tbl] = Table(tbl)
print(sga['dept'])

   TABLE: default.dept
   +-------+----------+
   |codigo |descricao |
   +-------+----------+
  0|      1|   Compras|
  1|      2|    Vendas|
  2|      3|   Suporte|
   +-------+----------+



## Alteração

O comando `ALTER TABLE` serve para alterar os atributos definidos para a tabela durante o processo de criação `CREATE TABLE`. Seus principais operadores servem para adição (`ADD`), modificação (`MODIFY`) tipo de dado, renomear (`RENAME`) um atributo ou ainda a remoção (`DROP`) de atributos existentes.

```sql
-- Adiciona uma nova coluna do tipo inteiro chamada budget
ALTER TABLE dept ADD (budget INTEGER)
```
```sql
-- Renomear a coluna para o nome em português e alterar seu tipo para ponto flutuante 
ALTER TABLE dept MODIFY (budget INTEGER) TO (orcamento FLOAT);
```
```sql
-- Remove a coluna 'descricao' da tabela
ALTER TABLE dept DROP descricao;
```

> Este sistema não prevê *concorrencia*, por isso podemos trabalhar sobre os objetos da SGA

In [9]:
sga['dept'].alter('ADD',{'budget':int}).save()
print(sga['dept'])

   TABLE: default.dept
   +-------+----------+-------+
   |codigo |descricao |budget |
   +-------+----------+-------+
  0|      1|   Compras|      0|
  1|      2|    Vendas|      0|
  2|      3|   Suporte|      0|
   +-------+----------+-------+



In [10]:
sga['dept'].alter('MODIFY',{'budget':int},{'orcamento':float}).save()
print(sga['dept'])

   TABLE: default.dept
   +-------+----------+----------+
   |codigo |descricao |orcamento |
   +-------+----------+----------+
  0|      1|   Compras|       0.0|
  1|      2|    Vendas|       0.0|
  2|      3|   Suporte|       0.0|
   +-------+----------+----------+



In [11]:
sga['dept'].alter('DROP',{'descricao':str}).save()
print(sga['dept'])

   TABLE: default.dept
   +-------+----------+
   |codigo |orcamento |
   +-------+----------+
  0|      1|       0.0|
  1|      2|       0.0|
  2|      3|       0.0|
   +-------+----------+



## Eliminação

Como o nome diz, o comando `DROP TABLE` eliminará o objeto do sistema. Ou seja, tanto o que está em memória quanto no sistema de arquivos e catalogo de dados.

```sql
DROP TABLE dept
```

In [12]:
sga['dept'].drop()
del sga['dept']